# Selekcja i ekstrakcja cech za pomocą scikit-learn

Ten notatnik pomoże Ci zapoznać się z metodami przetwarzania wstępnego danych w Pythonie. Po uzupełnieniu tego notatnika powinieneś:

+ zapoznać się klasą Pipeline,
+ wiedzieć jak znormalizować dane,
+ umieć uruchomić algorytm selekcji cech,
+ wiedzieć jak wykonać analizę PCA

Wszystkie algorytmy będziemy uruchamiać na jednym zbiorze danych: [Boston Housing Dataset](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html).

## Pipeline

Podczas przetwarzania wstępnego danych należy uważać, żeby nie korzystać z danych testowych. Wszakże jeśli nasz model zostanie uruchomiany na danych produkcyjnych będą to zupełnie nowe danych i nie będziemy zbyt dużo o nich wiedzieć w trakcie dokonywania predykcji.

Aby ułatwić użytkownikom biblioteki `scikit-learn` przetwarzanie wstępne z możliwością rozróżnienia danych treningowych od testowych, wprowadzono klasę `Pipeline`. Klasa `Pipeline` definiuje sekwencje kroków (transformacji), które należy wykonać na danych. Kolejnymi krokami pipeline'a mogą być:

- inżynieria nowych cech
- normalizacja danych
- usuwanie outlierów
- selekcja cech
- ekstrakcja cech
- uczenie klasyfikatora

Typowe pipeline'y są z reguły znacznie krótsze ;) Przydatna dokumentacja jak zwykle na stronie scikit-learn: [Pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

**Zad. 1: Załaduj wskazany zbiór danych i stwórz swój pierwszy pipeline. Pipeline powinien mieć dwa kroki: normalizację danych i uczenie regresora.**

In [1]:
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import numpy as np

SEED = 23

# 0. Zbiór danych
boston = load_boston()
X = boston["data"]
y = boston["target"]
names = boston["feature_names"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=SEED)

# 1. stwórz obiekt do normalizacji danych
# scaler = preprocessing.StandardScaler()

# 2. stwórz klasyfikator
# clf = ... może jakiś las?

# 3. stwórz Pipeline z dwoma krokami, kroki nazwij "scaler" i "clf" i niech zawierają obiekty scaler i clf
# pipe = ...

# 4. Odpal pipeline
# clf_fit = pipe.fit(X_train, y_train)
# y_true, y_pred = y_test, clf_fit.predict(X_test)
# rmse = np.sqrt(mean_squared_error(y_true, y_pred))
# print("RMSE: %.4f" % rmse)

`Pipeline` działa trochę jak klasyfikator połączony ze wstępnym przetwarzaniem. Można zatem podać stworzony przez siebie obiekt typu `Pipeline` jako parametr `GirdSearchCV`. Ale zanim do tego przejdziemy skupmy się na metodach selekcji cech.

## Selekcja cech

W poniższych sekcjach szybko spojrzymy na implementacje różnych metod selekcji cech w bibliotece `scikit-learn`. Warto zaznaczyć, że selekcja cech to bardzo szeroka działka naukowa i algorytmów jest multum. Stosunkowo niedawno pojawiła się biblioteka [`scikit-feature`](http://featureselection.asu.edu/), która rozszerza zbiór algorytmów dostępnych w `scikit-learn`. [`scikit-feature`](http://featureselection.asu.edu/) nie jest biblioteką, która jest oficjalnie wspierana przez ludzi tworzących `scikit-learn`, ale może to być dobre miejsce do poszukiwań implementacji mniej popularnych algorytmów.

### Metody filter

Zacznijmy od zbadania wariancji. Do tego przyda Ci się klasa [`Variancethreshold`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html). Jeśli wariancja jest zero (kolumna ma tylko jedną wartość) na pewno warto sprawdzić czy to nie jakiś błąd. Jeśli dana kolumna to po prostu stała, można ją z czystym sumieniem usunać. Mając rozeznanie w danych, można również ustawić minimalny próg zmienności.

**Zad. 2: Sprawdź czy w zbiorze danych są atrybuty o zmienności mniejszej niż 0.05.**

In [2]:
from sklearn.feature_selection import VarianceThreshold

# 1. Sprawdź ile atrybutów ma zbiór danych
# 2. Użyj metody fit_transform na obiekcie klasy VarianceThreshold
# 3. Sprawdź ile atrybutów ma przetransformowany zbiór danych

Inną prostą metodą jest badanie korelacji między zmiennymi a wartością przewidywaną. W `scikit-learn` służą do tego metody `chi2`, `f_classif` i `f_regression`. Ponieważ przewidujemy wartość ciągłą, sprawdźmy działanie tej ostatniej. Uwaga! Metody te oceniają każdą cechę osobno, dlatego cechy skorelowane będą podobnie ocenione.

**Zad. 3: Oceń atrybuty na podstawie ich korelacji z atrybutem decyzyjnym.**

In [3]:
from sklearn.feature_selection import f_regression

# 1. Odpalf_regression, aby uzyskać ocenę korelacji
# f_scores, p_values = ...

# 2. Wypisz wynik dla każdego atrybutu
# for i in range(len(names)):
#     print('{0}: {1:.2f} (p={2:.3f})'.format(names[i], f_scores[i], p_values[i]))

Znając korelację bądź inną wartość oceniającą atrybuty, możemy wybrać podzbioru najlepszych atrybutów. Do tego służą klasy `SelectKBest` i `SelectPercentile`



In [4]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

# 1. Stwórz obiekt SelectKBest z odpowiednimi parametrami

# 2. Stwórz pipeline z krokami scaler, selector i clf

# 3. Odpal pipeline i oceń predykcje tak jak to zrobiłeś w zadaniu 1.


**Zad. 5: Powtórz poprzednie zadanie wykorzystując tym razem miarę `mutual_info_regression`.**

In [5]:
# 1. Stwórz obiekt SelectKBest z odpowiednimi parametrami

# 2. Stwórz pipeline z krokami scaler, selector i clf

# 3. Odpal pipeline i oceń predykcje tak jak to zrobiłeś w zadaniu 1.


### Metody wrapper

Klasyczne metody typu wrapper nie są dostępne jako funkcje w `scikit-learn`. Można samemu zaimplementować brute-force, forward selction bądź backward selection lub... posiłkować się biblioteką [`mlxtend`](http://rasbt.github.io/mlxtend/) i zawartymi tam klasami `ExhaustiveFeatureSelector` (brute-force) i `SequentialFeatureSelector` (backward/foward selection). Biblioteka `mlxtend` zawiera wiele innych bardzo ciekawych rozszerzeń do `scikit-learn` (np. Stacking czy EnsembleVote) więc warto pamiętać o tej bibliotece.

Zamiast rozwodzić się nad klasycznymi metodami typu wrapper, które są bardzo kosztowne obliczeniowo, wypróbujmy algorytm RFE. Dla przypomnienia, algorytm RFE ocenia atrybuty a następnie usuwa najsłabszy z nich. Czynność ta jest powtarzana aż uzyskamy oczekiwaną liczbę atrybutów.


**Zad. 6: Skorzystaj z klasy [`RFECV`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html) aby znaleźć najlepszy podzbiór atrybutów. Użyj 10-krotnej oceny krzyżowej.**

In [6]:
from sklearn.feature_selection import RFECV

# 1. Odpal RFECV na danych treningowych

# 2. Wypisz ranking atrybutów
# for i in range(len(names)):
#     print('{0}: {1}'.format(names[i], selector.ranking_[i]))

Bardzo fajną hybrydą jest również algorytm Stability Selection. Jest to algorytm dość kosztowny ale łączy elementy interpretacji atrybutów oraz poprawiania trafności predykcji. Zainteresowani mogą zajrzeć do klas [`RandomizedLogisticRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RandomizedLogisticRegression.html) (klasyfikacja) i [`RandomizedLasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RandomizedLasso.html#sklearn.linear_model.RandomizedLasso) (regresja) w dokumentacji `scikit-learn`. Obie klasy są obecnie DEPRACATED, ale liczę że po prostu przeniosą je do modułu `feature_selection`.

### Metody embedded

Modele liniowe są od lat stosowane do określania ważności atrybutów. Modele liniowe bez regularyzacji potrafią wskazać ważność atrybutów jeśli dane nie są zbyt mocono zaszumione i atrybuty nie są ze sobą skorelowane.

Modele liniowe z regularyzacją radzą sobie lepiej z szumem i korelacją. Regularyzacja L1 (LASSO) usuwa atrybuty i może być stosowana do selekcji cech w celu poprawy trafności predykcji. Regularyzacja L2 (Ridge regression) jest bardzie stabilna, nie usuwa atrybutów i może być stosowania do oceny atrybutów w celach interpretacyjnych.

**Zad. 7: Naucz modele liniowe i sprawdź wagi przypisane kolejnym atrybutom by ocenić ich ważność. Nie zapomnij  oznormalizowaniu danych.**

In [7]:
from sklearn.linear_model import Lasso, Ridge

def pretty_print_linear(coefs, names = None, sort = False):
    if names is None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name) for coef, name in lst)

# 1. Odpal Lasso z alpha=0.3 w pipeline'ie z normalizacją
# lasso = ...
# lasso_pipe = ...
# lasso_pipe.fit(X_train, y_train)

# 2. Odpal Ridge z alpha=0.3 w pipeline'ie z normalizacją
# ridge = ...
# ridge_pipe = ...
# ridge_pipe.fit(X_train, y_train)

# 3. Wypisz uzyskane modele
# print("Lasso model: ", pretty_print_linear(lasso_pipe.steps[1][1].coef_, names, sort = True))
# print('------------------------------------------------------------------------------------')
# print("Ridge model: ", pretty_print_linear(ridge_pipe.steps[1][1].coef_, names, sort = True))

## Ekstrakcja cech

Na koniec zobaczymy jak w Pythonie policzyć PCA i na tej podstawie zmniejszyć liczbę atrybutów. Uwaga! Trzeba znormalizować dane przed analizą PCA, aby nie przecenić atrybutów o większym zakresie wartości i wten sposób nie wykonać gorszej transformacji.

**Zad. 8: Wykonaj analizę PCA zgodnie z poniższymi krokami.**

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# 1. Znormalizuj dane, odpal PCA i zobacz ile atrybutów potrzeba do wyjaśnienia zmienności w danych
# pca = Pipeline([('scaler', scaler), ('extractor', PCA())])
# pca.fit(X_train)

# plt.figure(1, figsize=(8, 6))
# plt.plot(pca.steps[1][1].explained_variance_, linewidth=2)
# plt.xlabel('n_components')
# plt.ylabel('explained_variance')


# 2. Stwórz nowy obiekt PCA i ogranicz liczbę cech do określonej na podstawie wykresu liczby

# 3. Stwórz pipeline z krokami scaler, extractor i clf

# 4. Odpal pipeline i oceń predykcje tak jak to zrobiłeś w zadaniu 1.


## Zadanie bonusowe

**Zad. 9*: Zaimplementuj grid search, który wypróbuje wiele metod selekcji cech (potencjalnie z różnymi parametrami) i wybierze najlepsze wstępne przetwarzanie danych.**